[<img width="200" alt="get in touch with Consensys Diligence" src="https://user-images.githubusercontent.com/2865694/56826101-91dcf380-685b-11e9-937c-af49c2510aa0.png">](https://diligence.consensys.net)<br/>
<sup>
[[  🌐  ](https://diligence.consensys.net)  [  📩  ](https://github.com/ConsenSys/vscode-solidity-doppelganger/blob/master/mailto:diligence@consensys.net)  [  🔥  ](https://consensys.github.io/diligence/)]
</sup><br/><br/>


# Hallucinate.sol - train & predict

## Initialization magic

We're training on google collab because it is much faster than on a normal pc. For this, we'll want to make the modules that download the source-samples and thrain the RNN available on google drive.

1. copy the files from https://github.com/tintinweb/hallucinate.sol to your personal google drive into `/MyDrive/collab/solidity-gen`
2. run the next two steps and provide your authenticator token 


In [1]:
%tensorflow_version 2.x

In [2]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/MyDrive/collab/solidity-gen')
!ls -lsat '/content/drive/MyDrive/collab/solidity-gen'
###
from soliditygen import *
import soliditygen
import importlib
importlib.reload(soliditygen)

Mounted at /content/drive
total 20
8 -rw------- 1 root root 8188 Nov 10 12:19 soliditygen.py
4 drwx------ 2 root root 4096 Nov 10 10:56 one_step
4 drwx------ 2 root root 4096 Nov 10 10:40 training_checkpoints
4 drwx------ 2 root root 4096 Nov 10 09:32 __pycache__


<module 'soliditygen' from '/content/drive/MyDrive/collab/solidity-gen/soliditygen.py'>

## Download Training Data

we'll download up to `maxfiles` or `maxlen` samples from https://github.com/tintinweb/smart-contract-inspector, clean them up, and concatename to one big sample file.

Note that we can do this multiple times to retrain the model.

In [3]:
# get trainingdata
trainingData = SolidityTrainer.get_training_data(maxfiles=3000, maxlen=15_000_000)

16384/8140 [============================================================] - 0s 0us/step


## Training

1. we're creating a new model with `embedding_dimension` and `rnn_units`.
2. then we pick shuffled samples from the input dataset. 
3. we output the model characteristics.
4. we start training the model for `epochs`.

this will take some time ☕

In [ ]:
print(f'Input characters: {trainingData.len}')

# The unique characters in the file
print(f'Vocab (unique chars): {len(trainingData.vocab)}')

# Take a look at the first 250 characters in text
print('First 250 chars:')
print("<----------------------------------------")
print(trainingData.text[:250])
print("---------------------------------------->")


####### - create model and train it
trainingData.newModel(embedding_dim=256, rnn_units=1024)

dataset = trainingData.getSampledDataset(seq_length=250, batch_size=64, buffer_size=10000)

trainingData.model.summary()

trainingData.train(dataset, epochs=15)


Length of text: 15001466 characters
140 unique characters
First 250 chars:
<--------------------
contract ERC20 {
  uint public totalSupply;
  function balanceOf(address who) constant returns (uint);
  function allowance(address owner, address spender) constant returns (uint);
  function transfer(address to, uint value) returns (bool ok);
  func
-------------------->
(64, 250, 141) # (batch_size, sequence_length, vocab_size)
Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  36096     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  144525    
                                                                 
Total params: 4,118,925
T

## Hallucinate

Predixt up to `num_characters=2000` starting from a specific set of seeds `['contract ']`.

This example should start hallucinating new solidity contracts starting from the `contract` keyword 💪.

In [29]:
## first run
start = time.time()
#
#
#
textOut = trainingData.predict(['contract '], 2000)
#
#
#
print(textOut, '\n\n' + '_'*80)
end = time.time()
print('\nRun time:', end - start)




contract AcceptsHalo3D playersCoin";
    uint32[4] public ICOperation;
    string public symbol;
    uint8 public constant decimals = 18;
    mapping (address => uint256) public balancesForAmount m;
        uint256 rebeaseSobISs;
        bool public founder = msg.sender;
        unlockTime = _unixTime;
    }
    function calculatePoohsBytes32(string extends) throw;
      _;
    }
    modifier onlyStronghands() {
        require(myTokens() > 0);
        _;
    }
    modifier contract_etherwowder() {
        require(msg.sender==oneTokenInFiatWei));
        _;
    }
    modifier notNull(address _to) {
        emit DividendTokenBalanceLedger_[msg.sender] = mintedAmount;
        emit TokensUsedRate(_tokenAddress].time);
        if (currentEthInvested < 0)
        {
            deposited += msg.value;
            tokens[0] = _senderToAmount;
            minerShare_ = SafeMath.ack(_eth, _amt[i]);
            }
        }
    }
    function executeTransaction(uint _required)
        public
    

## OPTIONAL - ReTrain the model

this can be used to incrementally re-train the model. This should allows us to continuously improve it with new data from the dataset

In [45]:
dataset = trainingData.getSampledDataset(seq_length=250, batch_size=64, buffer_size=10000)

trainingData.model.summary()

trainingData.train(dataset, epochs=10)

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  28160     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  112750    
                                                                 
Total params: 4,079,214
Trainable params: 4,079,214
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
920/920 [==============================] - 65s 67ms/step - loss: 0.3467
Epoch 2/10
920/920 [==============================] - 64s 68ms/step - loss: 0.3544
Epoch 3/10
920/920 [==============================] - 64s 69ms/step - loss: 0.3658
Epoch 4/10
920/920 [==============================] - 64s 68ms/

## Saving and Restoring the model

let's save the model, restore it from the saved state, and hallucinate more solidity code 🙌

In [11]:
trainingData.save_model("solidity_model")

INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


In [ ]:
###
trainingData.load_model("solidity_model") # reloaded

## 2nd run

start = time.time()
textOut = trainingData.predict(['contract ', 'contract ', 'abstract ', 'interface ', 'library '], 3000)
print(textOut, '\n\n' + '_'*80)
end = time.time()
print('\nRun time:', end - start)

## Export model to tensorflowjs

We want to make the model available in `tensorflowjs` so that we can easily generate solidity code from a webpage. For this we will have to convert the keras model from tensorflow format to tensorflowjs.

This notebook will then zip the model and provide it as a download.

In [ ]:
!pip install tensorflowjs
import tensorflowjs as tfjs
!ls -lsat .
!mkdir  solidity_model
one_step_model.save_weights('solidity_model/weights.h5')

#tf.saved_model.save(one_step_model, 'one_step_export_tin_2mb_20epochs_18min_training')
#tfjs.converters.convert_tf_saved_model("one_step_export_tin_2mb_20epochs_18min_training", "tfjs_out")
#tfjs.converters.save_keras_model(one_step_model, "./one_step_export_tin_2mb_20epochs_18min_training-js/")
!mkdir  solidity_model/js-out
!ls -lsat ./solidity_model/
!tensorflowjs_converter --input_format keras solidity_model/weights.h5 ./solidity_model/js-out/
!ls -lsat ./solidity_model/js-out/

from google.colab import files
!zip -r solidity_model.zip ./solidity_model/js-out
files.download('solidity_model.zip') 